In [1]:
import pandas as pd

csv_original = "results_of_nllb-200-distilled-600M.csv"
df_original = pd.read_csv(csv_original)
df_original.columns

Index(['source_texts', 'target_texts', 'references'], dtype='object')

In [2]:
source_texts = df_original["source_texts"]
target_texts = df_original["target_texts"]
references = df_original["references"]

In [3]:
from konlpy.tag import Okt
# from konlpy.tag import Mecab # Warning:KoNLPy’s Mecab() class is not supported on Windows machines.
from nltk.translate.bleu_score import corpus_bleu

# corpus_bleu 的结果更稳健，因为它是基于整个语料库的平均分数计算的，而 sentence_bleu 仅基于单个句子。
# 注：nltk3.8.1和python 3.12有点问题，要么降低python版本，要么按照以下链接的指导去修改bleu_score.py文件
# https://github.com/nltk/nltk/pull/3207
# https://github.com/nltk/nltk/blob/develop/nltk/translate/bleu_score.py
references_okt = [[Okt().morphs(reference)] for reference in references]
target_texts_okt = [Okt().morphs(target_text) for target_text in target_texts]
score = corpus_bleu(references_okt, target_texts_okt)
score

0.06025297433806512